In [24]:
import os
import pickle as pkl

import numpy as np
import pandas as pd

def load_pkl(file_path) :
    
    with open(file_path, 'rb') as f:
        
        df = pkl.load(f)
        
    return df

from transformers import BertTokenizerFast

tokenizer_for_load = BertTokenizerFast.from_pretrained('./model/BertTokenizer-6000-32000-vocab.txt'
                                                   , strip_accents=False
                                                   , lowercase=False)

def create_segments(inputs) :
    
    segment = []
    segment_num = 0
    
    for i, x in enumerate(inputs) :
        
        segment.append(segment_num)
        
        if x == 3 :
            segment_num+=1
            
    return np.array(segment)

Calling BertTokenizerFast.from_pretrained() with the path to a single file or url is deprecated


In [4]:
df = load_pkl('./dt/train_set-maksed-position-sample-10000.pkl')

In [44]:
idx = 532

In [45]:
segments = create_segments(df[idx]['x'])

In [46]:
word_df = pd.DataFrame({ "학습데이터" : tokenizer_for_load.convert_ids_to_tokens(df[idx]['x']),
                         "라벨링" :     tokenizer_for_load.convert_ids_to_tokens(df[idx]['label']),
                         "세그먼트_EMB" : segments})

word_df.loc[df[idx]['masked_position'], "MASK"] = 1
word_df["MASK"].fillna(0, inplace = True)

In [47]:
df[idx]['NSP']

[0, 1]

In [48]:
word_df.head(50)

,학습데이터,라벨링,세그먼트_EMB,MASK
0,[CLS],[CLS],0,0.0
1,[MASK],특히,0,1.0
2,비디오,비디오,0,0.0
3,아트,아트,0,0.0
4,##라는,##라는,0,0.0
5,새로운,새로운,0,0.0
6,[MASK],예술의,0,1.0
7,[MASK],범주,0,1.0
8,##를,##를,0,0.0
9,발전,발전,0,0.0


In [50]:
word_df.tail(50)

,학습데이터,라벨링,세그먼트_EMB,MASK
27,[SEP],[SEP],0,0.0
28,버,버,1,0.0
29,##기는,##기는,1,0.0
30,몇,몇,1,0.0
31,##억,##억,1,0.0
32,##베리,##베리,1,0.0
33,##나,##나,1,0.0
34,하는,하는,1,0.0
35,악마,악마,1,0.0
36,##의,##의,1,0.0
